In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

download=False

# Transform for normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # mean, std for RGB
])

# Download CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=download, transform=transform)
trainloader = DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

# Load the test set
testset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,            
    download=download,
    transform=transform
)

# Wrap in a DataLoader
testloader = DataLoader(
    testset,
    batch_size=64,
    shuffle=False,
    num_workers=2
)

In [5]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")
    else:
        return torch.device("cpu")

device = get_device()

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size=32*32*3, num_classes=10):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 32x32 -> 16x16
        x = self.pool(F.relu(self.conv2(x)))  # 16x16 -> 8x8
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [8]:
def train(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def test(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total


In [9]:
# MLP
mlp = MLP().to(device)
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(5):
    train(mlp, trainloader, optimizer, criterion)
    acc = test(mlp, testloader)
    print(f"MLP Epoch {epoch+1} - Test Accuracy: {acc:.4f}")

# CNN
cnn = CNN().to(device)
optimizer = torch.optim.Adam(cnn.parameters(), lr=1e-3)

for epoch in range(5):
    train(cnn, trainloader, optimizer, criterion)
    acc = test(cnn, testloader)
    print(f"CNN Epoch {epoch+1} - Test Accuracy: {acc:.4f}")


MLP Epoch 1 - Test Accuracy: 0.4741
MLP Epoch 2 - Test Accuracy: 0.4941
MLP Epoch 3 - Test Accuracy: 0.5068
MLP Epoch 4 - Test Accuracy: 0.5205
MLP Epoch 5 - Test Accuracy: 0.5152
CNN Epoch 1 - Test Accuracy: 0.6361
CNN Epoch 2 - Test Accuracy: 0.6882
CNN Epoch 3 - Test Accuracy: 0.7001
CNN Epoch 4 - Test Accuracy: 0.7296
CNN Epoch 5 - Test Accuracy: 0.7242


In [4]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Flatten CIFAR-10 for sklearn
X_train = trainset.data.reshape((len(trainset.data), -1))  # shape: [50000, 3072]
y_train = np.array(trainset.targets)

# Train a random forest
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
clf.fit(X_train, y_train)

# Test on test set (similarly reshaped)
X_test = testset.data.reshape((len(testset.data), -1))
y_test = np.array(testset.targets)
print("Accuracy:", clf.score(X_test, y_test))


Accuracy: 0.4626
